介绍了Numpy中的自定义通用函数<br>
ufunc本质上并非一种函数，而是一个表示函数的对象。该对象有四种方法（注意：这些方法仅对输入两个参数，输出一个参数的ufunc对象有效）

In [1]:
import numpy as np

### np.frompyfunc

该函数可以将一个python普通函数转换为适用于ndarray对象各元素的函数，从而实现广播

In [2]:
def basehandle(x):
    return x % 2 + 1

In [3]:
# 定义通用函数
"""
frompyfunc(func, nin, nout)
nin表述输入参数，nout表示输出值数量
"""
ufunc = np.frompyfunc(basehandle, 1, 1)
ufunc(np.arange(10))

array([1, 2, 1, 2, 1, 2, 1, 2, 1, 2], dtype=object)

### np.vectorize

将普通的python函数转变为矢量化操作的函数，可用于某个或多个ndarray的广播函数操作

In [4]:
# 定义一个标量函数，一个输入参数后面可以作用于一个ndarray
def compare(x, y):
    if x >= y:
        return x
    else:
        return y

In [5]:
ufunc2 = np.vectorize(compare)

In [6]:
ufunc2(np.arange(10), 5)

array([5, 5, 5, 5, 5, 5, 6, 7, 8, 9])

###  np.apply_along_axis

其功能可以类似于dataframe中的apply, 沿着某个轴做某种数学操作

In [7]:
def basehandle(x):
    return x % 2 + 1

In [9]:
np.apply_along_axis(basehandle, axis=0, arr=np.arange(10))

array([1, 2, 1, 2, 1, 2, 1, 2, 1, 2])

In [25]:
# 二维数组的操作
def get_mean(x):
    return (x[0] + x[-1])/2

In [26]:
arr = np.arange(9).reshape((3, 3))
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [27]:
np.apply_along_axis(get_mean, arr=arr, axis=0)

array([3., 4., 5.])

但这个函数不适用于如下要求：逐个元素进行逻辑判断。此时应该采用vectorize或frompyfunc

In [28]:
arr = np.random.randint(1, 10, 10)
arr

array([5, 8, 1, 5, 8, 2, 6, 8, 7, 6])

In [29]:
np.apply_along_axis(lambda x: 1 if x > 5 else -1, arr=arr, axis=0)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [30]:
# 采用np.vectorize
func = np.vectorize(lambda x: 1 if x > 5 else -1)
func(arr)

array([-1,  1, -1, -1,  1, -1,  1,  1,  1,  1])

In [31]:
# 采用frompyfunc
func2 = np.frompyfunc(lambda x: 1 if x > 5 else -1,1,1)
func2(arr)

array([-1, 1, -1, -1, 1, -1, 1, 1, 1, 1], dtype=object)

### ufunc对象的四类方法

以通用函数add为例，介绍其四种方法

#### add函数

In [15]:
x = np.array([1,2,3])
y = np.array([4,5,6])
np.add(x, y)

array([5, 7, 9])

#### add.reduce函数

类似于python的reduce函数，对计算结果进行归约.<br>
注意ufunc的reduce函数仅仅适用于二进制函数

In [17]:
np.add.reduce(x)

6

#### add.accumulate

In [20]:
# 相当于cumsum
np.add.accumulate(x)

array([1, 3, 6])

#### add.reduceat

In [22]:
"""
计算规则：
step1 : 0~3， 从索引0加到索引2， 即1+2+3
step2 : 3~2, 2<3 直接取索引3，即4
step3 : 2~6， 从索引2加到索引5， 即3+4+5+6
step4 : 索引6， 即7
"""
np.add.reduceat(np.arange(1, 8), [0, 3, 2, 6])

array([ 6,  4, 18,  7])

#### add.outer

返回一个数组，它的秩（rank）等于两个输入数组的秩的和。它会作用于两个 输入数组之间存在的所有元素对。

In [24]:
np.add.outer(np.arange(1, 4), np.arange(1, 5))

array([[2, 3, 4, 5],
       [3, 4, 5, 6],
       [4, 5, 6, 7]])